This notebook shows the allowed species for the jet

The flux, ${\cal N}$, of sublimated molecules leaving a directly illuminated patch of surface ice is
\begin{equation}
    {\cal N}=\frac{(1-p)Q(t)-\epsilon \sigma T_S^4}{\Delta H/N_{A}+\gamma kT_S}\, ,
\end{equation}
where $Q(t)$ is the local solar irradiance, $\epsilon$ is the surface emissivity, $\Delta H$ is the sublimation enthalpy, $\gamma$ is the adiabatic index of the escaping vapor, and $p$ is the surface albedo, which is controlled by the admixture of impurities in the ice. 

Sublimated molecules of mass, $\mu m_{\rm u}$, exit isotropically into the hemisphere associated with the local zenith. Momentum balance, combined with `Oumuamua's observed radial acceleration, implies that each normally illuminated unit area accelerates an equivalent column of net mass, $\delta m={\cal N}\,(\mu m_{\rm u} \gamma k T_{S})^{1/2}/2\vert \alpha \vert$ in the anti-solar direction, $-{\bf \hat{r}}$. The mass-to-surface-area ratio, $\eta$ for the oblate ellipsoid is given by
\begin{equation}
    \eta=\frac{2c\rho}{3(1+({c^2}/{e a^2})\tanh^{-1}e)},
\end{equation}
where $e=\sqrt{1-c^2/a^2}$ is the ellipsoid's eccentricity. 
Averaged over all orientations of the body, the equivalent directly illuminated area is $\xi=1/4$ of the total surface area \cite{Meltzer1949}. 'Oumuamua's rotational state, however, prevented isotropic time-averaged illumination, and in the SI, we analyze the jet-driven dynamics of 'Oumuamua's tumbling motion. Adopting the 6:6:1 shape model and the high signal-to-noise photometry obtained from 25 Oct 2017 through 1 Nov 2017\cite{Belton2018}, we find $\xi=0.31$. 

Combining the above relations gives an expression for the fraction, $f=f(p,\epsilon,\rho)$, of the surface covered with sublimating ice that is required to produce the observed radial acceleration

\begin{equation}
    f=f(p,\epsilon,\rho)=\frac{4}{3}\frac{(\Delta H/N_A+\gamma k T_S)}{((1-p)Q(t)-\epsilon \sigma T_S^4)}\frac{\rho c\vert \alpha(t) \vert}{(\mu m_{\rm u}\gamma k T_S)^{1/2}\xi(1+(c^2/ea^2)\tanh^{-1}e)}\,.
\end{equation}

In [9]:
# this function returns the number curly N of molecules per second per unit area in the jet
import numpy as np
from scipy import optimize
%matplotlib inline
import matplotlib.pyplot as plt

def get_number_persecond_perareav2(Temperature,deltaH_X,gamma,albedo,r0):
    #also input r in AU
    epsilon = 1.-albedo
    Lsunwatt = 3.848e26 #watts
    watt_to_erg = 1.e7 #ergs / watt
    Lsun = Lsunwatt*watt_to_erg
    AU=1.496e13#cm
    r=r0*AU

    flux = Lsun/(4.*np.pi*r*r)
    steffan_boltz= 5.670367e-5 #ergs per second per cm^2 per kelvin^4
    Avogadro=6.0221409e23
    boltzman = 1.381e-16 #erg / kelvin
    curlyN=(flux*(1.-albedo)-epsilon*steffan_boltz*Temperature**4.)/(deltaH_X/Avogadro+ gamma*boltzman*Temperature)
    return curlyN
def get_number_persecond_perarea(Temperature,deltaH_X,gamma,albedo):
    
    epsilon = 1.-albedo
    Lsunwatt = 3.848e26 #watts
    watt_to_erg = 1.e7 #ergs / watt
    Lsun = Lsunwatt*watt_to_erg
    AU=1.496e13#cm
    r=1.4*AU
    flux = Lsun/(4.*np.pi*r*r)
    steffan_boltz= 5.670367e-5 #ergs per second per cm^2 per kelvin^4
    Avogadro=6.0221409e23
    boltzman = 1.381e-16 #erg / kelvin
    curlyN=(flux*(1.-albedo)-epsilon*steffan_boltz*Temperature**4.)/(deltaH_X/Avogadro+ gamma*boltzman*Temperature)
    return curlyN
def get_deltam(Temperature,mass_X,curlyN,gamma):
    AU=1.496e13#cm
    r=1.4*AU
    A_ng_r = 4.92e-4*(r/AU)**(-2)
    boltzman = 1.381e-16 #erg / kelvin
    deltaM=curlyN*np.sqrt(gamma*boltzman*Temperature*mass_X)/A_ng_r/2.
    return deltaM

def get_deltamv2(Temperature,mass_X,curlyN,gamma,r0):
    AU=1.496e13#cm
    r=r0*AU
    A_ng_r = 4.92e-4*(r/AU)**(-2)
    boltzman = 1.381e-16 #erg / kelvin
    deltaM=curlyN*np.sqrt(gamma*boltzman*Temperature*mass_X)/A_ng_r/2.
    return deltaM
def get_eta(rho,volume,surface_area):
    return rho*volume/surface_area
def get_fraction_oblate(a0,b0,c0,Temperature,deltaH_X,mass_X,rho_X,gamma,albedo,xi):
    volume = 4./3.*np.pi*a0*a0*c0
    eccentricity = np.sqrt(1.-c0**2/a0**2)
    surface_area = 2.*np.pi*a0*a0*(1.+c0**2/a0**2/eccentricity*np.arctanh(eccentricity))
    #xi = 0.198
    #xi=0.25
    eta = get_eta(rho_X,volume,surface_area)
    curlyN=get_number_persecond_perarea(Temperature,deltaH_X,gamma,albedo)
    AU=1.496e13#cm

    #print get_number_persecond_perareav2(Temperature,deltaH_X,gamma,albedo,1.4)
    #print curlyN
    deltam=get_deltam(Temperature,mass_X,curlyN,gamma)
    #print deltam
    return eta/deltam/xi

def get_fraction_oblatev2(a0,b0,c0,Temperature,deltaH_X,mass_X,rho_X,gamma,albedo,r0):
    volume = 4./3.*np.pi*a0*a0*c0
    eccentricity = np.sqrt(1.-c0**2/a0**2)
    surface_area = 2.*np.pi*a0*a0*(1.+c0**2/a0**2/eccentricity*np.arctanh(eccentricity))
    xi = 0.198
    eta = get_eta(rho_X,volume,surface_area)
    curlyN=get_number_persecond_perareav2(Temperature,deltaH_X,gamma,albedo,r0)
    deltam=get_deltamv2(Temperature,mass_X,curlyN,gamma,r0)
    return eta/deltam/xi




In [11]:
#best fit oblate spheroid
a0=11500./2.
b0=11500./2.
c0=1900./2.
xi=0.25

rho_H2=0.08
albedo=0.1
#Temperature_H2=15.
Temperature_H2=6.
Joule_toErg = 1.e7
deltaH_H2 = 1000. *Joule_toErg#Jouels per mole
gamma=4./3.
mass_H2=2.*1.67e-24

print "for oblate H2, xi=0.25, f=",get_fraction_oblate(a0,b0,c0,Temperature_H2,deltaH_H2,mass_H2,rho_H2,gamma,albedo,xi)

mass_hydrogen=1.67e-24#grams

rho_NE=1.507 #grams/ cm^3
deltaH_NE = 1.9 *1.e3*Joule_toErg#Jouels per mole
mass_NE=20.18*mass_hydrogen#3.35e-23
Temperature_NE=8.
print "for oblate  Ne, xi=0.25 f=", get_fraction_oblate(a0,b0,c0,Temperature_NE,deltaH_NE,mass_NE,rho_NE,gamma,albedo,xi)

rho_N2=1.023 #grams/ cm^3
deltaH_N2 = 7.34 *1.e3*Joule_toErg#Jouels per mole
mass_N2=14.*mass_hydrogen*2.#2.*4.7e-23
Temperature_N2=23.

print "for oblate N2,  xi=0.25,f=",get_fraction_oblate(a0,b0,c0,Temperature_N2,deltaH_N2,mass_N2,rho_N2,gamma,albedo,xi)

rho_AR=1.753 #grams/ cm^3 #ref https://www.sciencedirect.com/science/article/pii/0375960167906561
deltaH_AR = 7.79 *1.e3*Joule_toErg#Jouels per mole
mass_AR=mass_hydrogen*39.948
Temperature_AR=27.5

print "for oblate AR, xi=0.25, f=", get_fraction_oblate(a0,b0,c0,Temperature_AR,deltaH_AR,mass_AR,rho_AR,gamma,albedo,xi)

rho_O2=1.525 #grams/ cm^3 #ref https://www.sciencedirect.com/science/article/pii/0375960167906561
deltaH_O2 = 9.26 *1.e3*Joule_toErg#Jouels per mole
mass_O2=mass_hydrogen*(15.9994*2.)
Temperature_O2=29.

print "for oblate O2, xi=0.25, f=",get_fraction_oblate(a0,b0,c0,Temperature_O2,deltaH_O2,mass_O2,rho_O2,gamma,albedo,xi)


rho_KR=3.04 #grams/ cm^3
deltaH_KR = 11.54 *1.e3*Joule_toErg#Jouels per mole
mass_KR=83.8*mass_hydrogen#2.*4.7e-23
Temperature_KR=38.5

print "for oblate KR, xi=0.25, f=",get_fraction_oblate(a0,b0,c0,Temperature_KR,deltaH_KR,mass_KR,rho_KR,gamma,albedo,xi)

rho_XE=3.706 #grams/ cm^3
deltaH_XE = 15.79 *1.e3*Joule_toErg#Jouels per mole
mass_XE=131.3*mass_hydrogen#2.*4.7e-23

Temperature_XE=50.

print "for oblate XE, xi=0.25, f=",get_fraction_oblate(a0,b0,c0,Temperature_XE,deltaH_XE,mass_XE,rho_XE,gamma,albedo,xi)

#https://webbook.nist.gov/cgi/cbook.cgi?ID=C630080&Mask=4
rho_CO=1.56 #grams/ cm^3
deltaH_CO = 8.1 *1.e3*Joule_toErg#Jouels per mole
mass_CO=(12.+15.999)*mass_hydrogen#2.*4.7e-23
Temperature_CO=60.
#print "for oblate CO, f=",get_fraction_oblate(a0,b0,c0,Temperature_CO,deltaH_CO,mass_CO,rho_CO,gamma,albedo)



rho_CO2=1.56 #grams/ cm^3
deltaH_CO2 = 28.84 *1.e3*Joule_toErg#Jouels per mole
mass_CO2=(12.+2.*15.999)*mass_hydrogen#2.*4.7e-23
Temperature_CO2=80.
print "for oblate CO2, xi=0.25, f=",get_fraction_oblate(a0,b0,c0,Temperature_CO2,deltaH_CO2,mass_CO2,rho_CO2,gamma,albedo,xi)




mass_H2O = 3.e-23
DeltaH_Ice = 54.46*1.e3 #molar heat of water in J/mol
deltaH_H2O = DeltaH_Ice*Joule_toErg
rho_H2O = .82
Temperature_H2O=155.

print "for oblate H2O, xi=0.25, f=", get_fraction_oblate(a0,b0,c0,Temperature_H2O,deltaH_H2O,mass_H2O,rho_H2O,gamma,albedo, xi=0.25)




 for oblate H2, xi=0.25, f= 0.044174749123019046
for oblate  Ne, xi=0.25 f= 0.42303846226991826
for oblate N2,  xi=0.25,f= 0.5491276040800424
for oblate AR, xi=0.25, f= 0.767896099844472
for oblate O2, xi=0.25, f= 0.8603325682983457
for oblate KR, xi=0.25, f= 1.148898241282887
for oblate XE, xi=0.25, f= 1.3414643381666795
for oblate CO2, xi=0.25, f= 1.406577264366622
for oblate H2O, xi=0.25, f= 1.6427553802803088


In [14]:
#best fit oblate spheroid
a0=11500./2.
b0=11500./2.
c0=1900./2.
xi = 0.198

print "for oblate H2, xi=0.2, f=",get_fraction_oblate(a0,b0,c0,Temperature_H2,deltaH_H2,mass_H2,rho_H2,gamma,albedo,xi)
print "for oblate  Ne, xi=0.2 f=", get_fraction_oblate(a0,b0,c0,Temperature_NE,deltaH_NE,mass_NE,rho_NE,gamma,albedo,xi)
print "for oblate N2,  xi=0.2,f=",get_fraction_oblate(a0,b0,c0,Temperature_N2,deltaH_N2,mass_N2,rho_N2,gamma,albedo,xi)
print "for oblate AR, xi=0.2, f=", get_fraction_oblate(a0,b0,c0,Temperature_AR,deltaH_AR,mass_AR,rho_AR,gamma,albedo,xi)
print "for oblate O2, xi=0.2, f=",get_fraction_oblate(a0,b0,c0,Temperature_O2,deltaH_O2,mass_O2,rho_O2,gamma,albedo,xi)
print "for oblate KR, xi=0.2, f=",get_fraction_oblate(a0,b0,c0,Temperature_KR,deltaH_KR,mass_KR,rho_KR,gamma,albedo,xi)
print "for oblate XE, xi=0.2, f=",get_fraction_oblate(a0,b0,c0,Temperature_XE,deltaH_XE,mass_XE,rho_XE,gamma,albedo,xi)
print "for oblate CO2, xi=0.2, f=",get_fraction_oblate(a0,b0,c0,Temperature_CO2,deltaH_CO2,mass_CO2,rho_CO2,gamma,albedo,xi)
print "for oblate H2O, xi=0.2, f=", get_fraction_oblate(a0,b0,c0,Temperature_H2O,deltaH_H2O,mass_H2O,rho_H2O,gamma,albedo, xi)





for oblate H2, xi=0.2, f= 0.05577619838765031
for oblate  Ne, xi=0.2 f= 0.5341394725630281
for oblate N2,  xi=0.2,f= 0.6933429344444979
for oblate AR, xi=0.2, f= 0.9695657826319091
for oblate O2, xi=0.2, f= 1.086278495326194
for oblate KR, xi=0.2, f= 1.4506290925288976
for oblate XE, xi=0.2, f= 1.6937681037458074
for oblate CO2, xi=0.2, f= 1.7759813944023002
for oblate H2O, xi=0.2, f= 2.074186086212511


In [15]:
#best fit oblate spheroid
a0=32400./2.
b0=4200./2.
c0=4200./2.
def get_fraction_prolate(a0,b0,c0,Temperature,deltaH_X,mass_X,rho_X,gamma,albedo):
    volume = 4./3.*np.pi*a0*b0*c0
    eccentricity = np.sqrt(1.-c0**2/a0**2)
    surface_area = 2.*np.pi*c0*c0*(1.+a0/c0/eccentricity*np.arcsin(eccentricity))
    xi = 0.13
    #xi=0.25
    eta = get_eta(rho_X,volume,surface_area)
    curlyN=get_number_persecond_perarea(Temperature,deltaH_X,gamma,albedo)
    deltam=get_deltam(Temperature,mass_X,curlyN,gamma)
    return eta/deltam/xi



print "for prolate H2,xi = 0.13 f=",get_fraction_prolate(a0,b0,c0,Temperature_H2,deltaH_H2,mass_H2,rho_H2,gamma,albedo)

print "for prolate Ne,xi = 0.13 f=", get_fraction_prolate(a0,b0,c0,Temperature_NE,deltaH_NE,mass_NE,rho_NE,gamma,albedo)

print "for prolate N2,xi = 0.13 f=",get_fraction_prolate(a0,b0,c0,Temperature_N2,deltaH_N2,mass_N2,rho_N2,gamma,albedo)
print "for prolate AR,xi = 0.13 f=", get_fraction_prolate(a0,b0,c0,Temperature_AR,deltaH_AR,mass_AR,rho_AR,gamma,albedo)

print "for prolate O2,xi = 0.13 f=",get_fraction_prolate(a0,b0,c0,Temperature_O2,deltaH_O2,mass_O2,rho_O2,gamma,albedo)

print "for prolate KR,xi = 0.13 f=",get_fraction_prolate(a0,b0,c0,Temperature_KR,deltaH_KR,mass_KR,rho_KR,gamma,albedo)


print "for prolate XE,xi = 0.13 f=",get_fraction_prolate(a0,b0,c0,Temperature_XE,deltaH_XE,mass_XE,rho_XE,gamma,albedo)

print "for prolate CO2,xi = 0.13 f=",get_fraction_prolate(a0,b0,c0,Temperature_CO2,deltaH_CO2,mass_CO2,rho_CO2,gamma,albedo)

print "for prolate H2O,xi = 0.13 f=", get_fraction_prolate(a0,b0,c0,Temperature_H2O,deltaH_H2O,mass_H2O,rho_H2O,gamma,albedo)





for prolate H2,xi = 0.13 f= 0.12681697632034117
for prolate Ne,xi = 0.13 f= 1.2144598377429616
for prolate N2,xi = 0.13 f= 1.5764368501456034
for prolate AR,xi = 0.13 f= 2.2044779753987047
for prolate O2,xi = 0.13 f= 2.469844811968748
for prolate KR,xi = 0.13 f= 3.2982598419179405
for prolate XE,xi = 0.13 f= 3.8510790572711544
for prolate CO2,xi = 0.13 f= 4.038005402841351
for prolate H2O,xi = 0.13 f= 4.716026107606404


In [16]:
#best fit oblate spheroid
a0=32400./2.
b0=4200./2.
c0=4200./2.
def get_fraction_prolate(a0,b0,c0,Temperature,deltaH_X,mass_X,rho_X,gamma,albedo):
    volume = 4./3.*np.pi*a0*b0*c0
    eccentricity = np.sqrt(1.-c0**2/a0**2)
    surface_area = 2.*np.pi*c0*c0*(1.+a0/c0/eccentricity*np.arcsin(eccentricity))
    xi = 0.13
    xi=0.25
    eta = get_eta(rho_X,volume,surface_area)
    curlyN=get_number_persecond_perarea(Temperature,deltaH_X,gamma,albedo)
    deltam=get_deltam(Temperature,mass_X,curlyN,gamma)
    return eta/deltam/xi



print "for prolate H2,xi = 0.25 f=",get_fraction_prolate(a0,b0,c0,Temperature_H2,deltaH_H2,mass_H2,rho_H2,gamma,albedo)

print "for prolate Ne,xi = 0.25 f=", get_fraction_prolate(a0,b0,c0,Temperature_NE,deltaH_NE,mass_NE,rho_NE,gamma,albedo)

print "for prolate N2,xi = 0.25 f=",get_fraction_prolate(a0,b0,c0,Temperature_N2,deltaH_N2,mass_N2,rho_N2,gamma,albedo)
print "for prolate AR,xi = 0.25 f=", get_fraction_prolate(a0,b0,c0,Temperature_AR,deltaH_AR,mass_AR,rho_AR,gamma,albedo)

print "for prolate O2,xi = 0.25 f=",get_fraction_prolate(a0,b0,c0,Temperature_O2,deltaH_O2,mass_O2,rho_O2,gamma,albedo)

print "for prolate KR,xi = 0.25 f=",get_fraction_prolate(a0,b0,c0,Temperature_KR,deltaH_KR,mass_KR,rho_KR,gamma,albedo)


print "for prolate XE,xi = 0.25 f=",get_fraction_prolate(a0,b0,c0,Temperature_XE,deltaH_XE,mass_XE,rho_XE,gamma,albedo)

print "for prolate CO2,xi = 0.25 f=",get_fraction_prolate(a0,b0,c0,Temperature_CO2,deltaH_CO2,mass_CO2,rho_CO2,gamma,albedo)

print "for prolate H2O,xi = 0.25 f=", get_fraction_prolate(a0,b0,c0,Temperature_H2O,deltaH_H2O,mass_H2O,rho_H2O,gamma,albedo)






for prolate H2,xi = 0.25 f= 0.0659448276865774
for prolate Ne,xi = 0.25 f= 0.6315191156263401
for prolate N2,xi = 0.25 f= 0.8197471620757139
for prolate AR,xi = 0.25 f= 1.1463285472073266
for prolate O2,xi = 0.25 f= 1.284319302223749
for prolate KR,xi = 0.25 f= 1.715095117797329
for prolate XE,xi = 0.25 f= 2.0025611097810003
for prolate CO2,xi = 0.25 f= 2.0997628094775025
for prolate H2O,xi = 0.25 f= 2.4523335759553304
